In [ ]:
!pip install accelerate
!pip install -q --upgrade langchain
!pip install -q --upgrade langchain-openai
!pip install -q --upgrade langchain_community
!pip install -q transformers
!pip install -q faiss-gpu
!pip install -q pandas
!pip install faiss-cpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
# !pip uninstall -y numpy
# !pip install numpy==1.26.4

In [ ]:
!pip uninstall numpy pandas -y
!pip install numpy pandas --upgrade --force-reinstall


Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-

## 벡터 db 생성

In [ ]:
import os
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import math

# 1. CSV 불러와서 Document로 변환
def load_documents(path="merged_rag_data.csv"):
    df = pd.read_csv(path, encoding="utf-8")
    docs = []
    for i, row in df.iterrows():
        title = str(row.get("title", "")).strip()
        content = str(row.get("content", "")).strip()

        if content:
            full_text = f"{title} - {content}" if title else content
            doc = Document(
                page_content=full_text,
                metadata={
                    "source": f"doc_{i}",
                    "title": title
                }
            )
            docs.append(doc)

    print(f"총 문서 수: {len(docs)}")
    return docs

# 2. 문서 분할 (chunking)
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    return splitter.split_documents(documents)

# 3. 임베딩 & 벡터 DB 생성 및 저장
def build_vector_db(documents, save_path="openai_faiss_db"):
    print("OpenAI 임베딩 실행 시작")
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectordb = FAISS.from_documents(documents, embedding=embeddings)
    vectordb.save_local(save_path)
    print(f"벡터 DB 저장 완료")

def build_vector_db(documents, save_path="openai_faiss_db", batch_size=300):
    print("OpenAI 임베딩 실행 시작")
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    texts = [doc.page_content for doc in documents]
    metadatas = [doc.metadata for doc in documents]

    text_embedding_pairs = []

    total_batches = math.ceil(len(texts) / batch_size)

    for i in range(total_batches):
        start = i * batch_size
        end = start + batch_size
        batch_texts = texts[start:end]
        batch_metadatas = metadatas[start:end]

        try:
            batch_vectors = embeddings.embed_documents(batch_texts)
        except Exception as e:
            print(f"임베딩 실패 (배치 {i+1}/{total_batches}): {e}")
            continue

        for text, vector, metadata in zip(batch_texts, batch_vectors, batch_metadatas):
            text_embedding_pairs.append((text, vector, metadata))

        print(f"배치 {i+1}/{total_batches} 임베딩 완료")

    # 직접 벡터 저장
    texts, vectors, metadatas = zip(*text_embedding_pairs)
    vectordb = FAISS.from_embeddings(
        text_embeddings=list(zip(texts, vectors)),
        embedding=embeddings,
        metadatas=metadatas
    )
    vectordb.save_local(save_path)
    print(f"벡터 DB 저장 완료")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw_docs = load_documents("/content/drive/MyDrive/merged_rag_data.csv")
split_docs = split_documents(raw_docs)

총 문서 수: 1609


In [ ]:
print(f"전체 문서 수: {len(split_docs)}")
print(f"가장 긴 문서 길이 (문자 수): {max(len(doc.page_content) for doc in split_docs)}")

전체 문서 수: 6500
가장 긴 문서 길이 (문자 수): 512


In [ ]:
from google.colab import userdata

# api_key = userdata.get("")
# if api_key is None:
#     raise ValueError("OPENAI_API_KEY 설정X")
# os.environ["OPENAI_API_KEY"] = api_key

os.environ["OPENAI_API_KEY"] = ""


In [ ]:
build_vector_db(split_docs, save_path="openai_faiss_db")

OpenAI 임베딩 실행 시작
배치 1/22 임베딩 완료
배치 2/22 임베딩 완료
배치 3/22 임베딩 완료
배치 4/22 임베딩 완료
배치 5/22 임베딩 완료
배치 6/22 임베딩 완료
배치 7/22 임베딩 완료
배치 8/22 임베딩 완료
배치 9/22 임베딩 완료
배치 10/22 임베딩 완료
배치 11/22 임베딩 완료
배치 12/22 임베딩 완료
배치 13/22 임베딩 완료
배치 14/22 임베딩 완료
배치 15/22 임베딩 완료
배치 16/22 임베딩 완료
배치 17/22 임베딩 완료
배치 18/22 임베딩 완료
배치 19/22 임베딩 완료
배치 20/22 임베딩 완료
배치 21/22 임베딩 완료
배치 22/22 임베딩 완료
벡터 DB 저장 완료


## 저장된 벡터 db로 문서 유사도 검색 결과

In [ ]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = FAISS.load_local("openai_faiss_db", embedding_model, allow_dangerous_deserialization=True)

In [ ]:
query = "강아지가 계속 짖는 이유가 궁금해요"
results = vectordb.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n📄 결과 {i+1}")
    print("내용:", doc.page_content)
    print("메타데이터:", doc.metadata)

## 모델 추론 과정에 RAG구축

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


In [ ]:
# === 1. 모델 & 토크나이저 로드 ===
model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [ ]:
# === 2. 시스템 프롬프트 설정 ===
system_prompt = """당신은 반려견 행동 문제를 상담해주는 전문가입니다.

상담의 목적은, 단순한 정보 제공이 아니라 **사용자의 상황을 정확히 이해한 뒤, 그에 맞는 맞춤형 해결책을 제시하는 것**입니다.

아래의 상담 구조를 반드시 따르세요:

1. 사용자가 고민을 입력하면, 그 고민의 원인을 '추측'하거나 '일반화'하지 말고, **반드시 추가 질문을 통해 정보를 더 수집**하세요.
2. **반려견의 품종 정보를 고려하여** 행동 특성, 기질, 환경 민감도를 분석에 반영하세요.
3. 질문은 1개로 짧게, **사용자가 답하기 쉽도록 구체적이고 상황 중심적으로** 만들어야 합니다.
4. 추가 질문이 1-2번 이루어졌으면, **해결책을 1가지로 요약해서 제시**하세요.
   (여러 해결책을 나열하거나 조건 없이 모두 설명하지 마세요.)
5. 모든 답변은 **공감 → 질문 또는 분석 → 해결책 제시**의 흐름을 따라야 합니다.

- 상담의 시작은 항상 보호자의 감정을 공감하는 문장으로 시작하세요.
- 문장의 시작에는 다음 형식을 사용하세요:
  **"안녕하세요! (반려견 이름) 보호자님! (반려견 이름)의 (고민 내용) 때문에 고민이 많으시겠어요."**

❗절대 하지 말아야 할 것:
- 고민 입력만으로 바로 해결책을 나열하지 마세요.
- 질문 없이 바로 솔루션을 제시하지 마세요.
- 같은 내용을 반복하거나 불필요하게 장황하게 설명하지 마세요.
"""

messages = [{"role": "system", "content": system_prompt}]


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# === 3. FAISS 벡터 DB 로딩 ===
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = FAISS.load_local("openai_faiss_db", embedding_model, allow_dangerous_deserialization=True)


In [ ]:
# === 4. 대화 루프 시작 ===
print("🐾 반려견 행동 상담 챗봇입니다. '완료'를 입력하면 종료됩니다.")
dog_breed = input("\n🐶 반려견 종을 입력하세요: ").strip()
dog_name = input("\n🐶 반려견의 이름을 입력하세요: ").strip()
while True:
    user_input = input("\n🧑 사용자 고민: ").strip()
    if "완료" in user_input:
        print("\n✅ 대화를 종료합니다.")
        break

   # === ✅ RAG: 관련 문서 검색
    retrieved_docs = vectordb.similarity_search(user_input, k=3)

    print("\n📄 [검색된 문서 요약]")
    for i, doc in enumerate(retrieved_docs):
        # print(f"\n🔎 문서 {i+1}:\n{doc.page_content[:300]}...")  # 필요 시 300자 이상도 출력 가능
        if doc.metadata:
            print(f"   ⤷ 출처: {doc.metadata}")

    retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # === ✅ RAG context 포함한 사용자 메시지 구성
    user_message = f"관련 정보:\n{retrieved_context}\n\n사용자 반려견 정보:\n견종: {dog_breed}\n이름: {dog_name}\n\n질문:\n{user_input}"
    messages.append({"role": "user", "content": user_message})

    # === 5. ChatML 템플릿 적용
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True
    )

    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            temperature=0.6,
            top_p=0.95,
            top_k=20,
            do_sample=True
        )

    output_ids = outputs[0][inputs.input_ids.shape[-1]:].tolist()

    # === 6. 사고모드 </think> 분리
    try:
        end_token_id = 151668  # </think>
        index = len(output_ids) - output_ids[::-1].index(end_token_id)
    except ValueError:
        index = 0

    thinking = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()

    # === 7. 응답 출력 및 메시지 추가
    # print(f"\n🧠 [thinking]: {thinking}")
    print(f"🤖 [assistant]: {content}")

    messages.append({"role": "assistant", "content": content})